In [1]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout
import pandas as pd

In [2]:
# real test case

In [26]:
AWSEM_xml = "/Users/weilu/openmmawsem/awsem_heme.xml"
pre = "/Users/weilu/Research/server/may_week3_2020/hemoglobin/setups/debug_displacement_of_N/"
fileLocation = "/Users/weilu/Research/server/may_week3_2020/hemoglobin/setups/four_chains/5x2r-openmmawsem.pdb"
fileLocation = "/Users/weilu/Research/server/may_week3_2020/hemoglobin/setups/debug_displacement_of_N/5x2r-openmmawsem.pdb"
pdb = PDBFile(fileLocation)
# pdb = PDBFile("/Users/weilu/Research/server/jan_2020/include_small_molecular/my_ABCDE_with_ligand/my_ABCDE_with_ligand-openmmawsem.pdb")

In [27]:
def compute_dis(pos1, pos2):
    dis = pos1 - pos2
    dis = dis.value_in_unit(nanometer)
    r = (dis[0]**2 + dis[1]**2 + dis[2]**2)**0.5
    return r

In [28]:
res_list = list(pdb.topology.residues())
atom_list = list(pdb.topology.atoms())
protein_resNames = ["NGP", "IGL", "IPR", "NTER", "CTER"]
DNA_resNames = ["DA", "DC", "DT", "DG"]
protein_res_list = []
DNA_res_list = []
ligand_res_list = []
for res in res_list:
    if res.name in protein_resNames:
        protein_res_list.append(res)
    elif res.name in DNA_resNames:
        DNA_res_list.append(res)
    else:
        ligand_res_list.append(res)

protein_atom_list = []
DNA_atom_list = []
ligand_atom_list = []
for atom in atom_list:
    if atom.residue.name in protein_resNames:
        protein_atom_list.append(atom)
    elif atom.residue.name in DNA_resNames:
        DNA_atom_list.append(atom)
    else:
        ligand_atom_list.append(atom)

In [29]:
forcefield = ForceField(AWSEM_xml)
[templates, names] = forcefield.generateTemplatesForUnmatchedResidues(pdb.topology)
for a in templates:
    for a1 in a.atoms:
        if a1.element.symbol == "C":
            a1.type = "CA"
        else:
            a1.type = a1.element.symbol
        # a1.type = a1.element.symbol
    forcefield.registerResidueTemplate(a)
system = forcefield.createSystem(pdb.topology)

In [30]:
info = []
for ligand_atom in ligand_atom_list:
    pos1 = pdb.positions[ligand_atom.index]
    for protein_atom in protein_atom_list:
        if protein_atom.name != "CB":
            continue
        protein_chain = protein_atom.residue.chain.id
        pos2 = pdb.positions[protein_atom.index]
        r = compute_dis(pos1, pos2)
        if r < 0.65:
            # print(ligand_atom, protein_atom, "in contact", r)
            info.append([ligand_atom.index, protein_atom.index, r, protein_chain])

In [31]:
data = pd.DataFrame(info, columns=["Ligand_atom_index", "Protein_atom_index", "r", "Protein_chain"])

In [32]:
data.to_csv(f"{pre}/het_protein_bonds.csv", index=False)

In [33]:
data

,Ligand_atom_index,Protein_atom_index,r,Protein_chain
0,3340,353,0.567563,A
1,3340,502,0.562452,A
2,3341,330,0.646263,A
3,3341,347,0.600552,A
4,3341,496,0.586840,A
...,...,...,...,...
548,3508,2724,0.628420,D
549,3508,3046,0.585422,D
550,3509,3046,0.563507,D
551,3510,3046,0.524599,D


In [34]:
# data = pd.read_csv(f"{pre}/het_protein_bonds.csv")

In [35]:
res_list = list(pdb.topology.residues())
atom_list = list(pdb.topology.atoms())
protein_resNames = ["NGP", "IGL", "IPR", "NTER", "CTER"]
DNA_resNames = ["DA", "DC", "DT", "DG"]
protein_res_list = []
DNA_res_list = []
ligand_res_list = []
for res in res_list:
    if res.name in protein_resNames:
        protein_res_list.append(res)
    elif res.name in DNA_resNames:
        DNA_res_list.append(res)
    else:
        ligand_res_list.append(res)

protein_atom_list = []
DNA_atom_list = []
ligand_atom_list = []
for atom in atom_list:
    if atom.residue.name in protein_resNames:
        protein_atom_list.append(atom)
    elif atom.residue.name in DNA_resNames:
        DNA_atom_list.append(atom)
    else:
        ligand_atom_list.append(atom)
ligand_res_list



[<Residue 570 (HNI) of chain 4>,
 <Residue 571 (HEM) of chain 5>,
 <Residue 572 (HEM) of chain 6>,
 <Residue 573 (HNI) of chain 7>]

In [36]:
info = []
for res in ligand_res_list:
    atoms = list(res.atoms())
    n_atoms = len(atoms)
    for i in range(n_atoms):
        atom1 = atoms[i]
        pos1 = pdb.positions[atom1.index]
        for j in range(i+1, n_atoms):
            atom2 = atoms[j]
            pos2 = pdb.positions[atom2.index]
            dis = pos1 - pos2
            dis = dis.value_in_unit(nanometer)
            r = (dis[0]**2 + dis[1]**2 + dis[2]**2)**0.5
            # print(atom1.index, atom2.index, round(r, 3))
            info.append([res.name, atom1.index, atom2.index, atom1.name, atom2.name, atom1.element.symbol, atom2.element.symbol, round(r, 3)])

data = pd.DataFrame(info, columns=["Name", "Atom1", "Atom2", "Name1", "Name2", "Symbol1", "Symbol2", "r"])
data

,Name,Atom1,Atom2,Name1,Name2,Symbol1,Symbol2,r
0,HNI,3340,3341,NI,CHA,Ni,C,0.336
1,HNI,3340,3342,NI,CHB,Ni,C,0.336
2,HNI,3340,3343,NI,CHC,Ni,C,0.335
3,HNI,3340,3344,NI,CHD,Ni,C,0.337
4,HNI,3340,3345,NI,NA,Ni,N,0.193
...,...,...,...,...,...,...,...,...
3607,HNI,3508,3510,CBD,O1D,C,O,0.240
3608,HNI,3508,3511,CBD,O2D,C,O,0.250
3609,HNI,3509,3510,CGD,O1D,C,O,0.122
3610,HNI,3509,3511,CGD,O2D,C,O,0.135


In [37]:
data.to_csv(f"{pre}/het_frag.csv", index=False)

In [ ]:
info = []
for res in ligand_res_list:
    atoms = list(res.atoms())
    n_atoms = len(atoms)
    for i in range(n_atoms):
        atom1 = atoms[i]
        pos1 = pdb.positions[atom1.index]
        for j in range(i+1, n_atoms):
            atom2 = atoms[j]
            pos2 = pdb.positions[atom2.index]
            r = compute_dis(pos1, pos2)
            # print(atom1.index, atom2.index, round(r, 3))
            info.append([res.name, atom1.index, atom2.index, atom1.name, atom2.name, atom1.element.symbol, atom2.element.symbol, round(r, 3)])
            
            

data = pd.DataFrame(info, columns=["Name", "Atom1", "Atom2", "Name1", "Name2", "Symbol1", "Symbol2", "r"])

In [33]:
data.dtypes

Ligand_atom_index       int64
Protein_atom_index      int64
r                     float64
Protein_chain          object
dtype: object

In [34]:
protein_atom_list[0]

<Atom 0 (CA) of chain 0 residue 0 (NGP)>

In [23]:
res.

'CLR'

In [123]:
atom.name

'C18'

In [114]:
d = pdb.positions[atom.index]

In [121]:
atom = ligand_atom_list[0]

In [ ]:
atom.index

In [91]:
a = list(pdb.getTopology().bonds())

In [100]:
a = ligand_atom_list[0]

In [103]:
ligand_atoms_index_list = [atom.index for atom in ligand_atom_list]
protein_atoms_index_list = [atom.index for atom in protein_atom_list]

In [106]:
a = protein_atom_list[0]

0

In [96]:
atom1 = a[0].atom1

In [97]:
atom1.index

3548

In [98]:
atom1.id

'31351'

In [128]:
pdb.getTopology()

<Topology; 13 chains, 612 residues, 3760 atoms, 220 bonds>

In [4]:
forcefield = ForceField(AWSEM_xml)

In [5]:
[templates, names] = forcefield.generateTemplatesForUnmatchedResidues(pdb.topology)

In [6]:
for a in templates:
    for a1 in a.atoms:
        if a1.element.symbol == "C":
            a1.type = "CA"
        else:
            a1.type = a1.element.symbol
        # a1.type = a1.element.symbol
    forcefield.registerResidueTemplate(a)

In [7]:
system = forcefield.createSystem(pdb.topology)

In [8]:
a = templates[0]

In [61]:
names

[<Residue 604 (OLC) of chain 5>, <Residue 606 (CLR) of chain 7>]

[<Residue 570 (HNI) of chain 4>,
 <Residue 571 (HEM) of chain 5>,
 <Residue 572 (HEM) of chain 6>,
 <Residue 573 (HNI) of chain 7>]

In [15]:
positions = pdb.positions

In [46]:
atoms = list(res.atoms())

In [48]:
atom = atoms[0]

In [49]:
atom.index

3548

In [68]:
atom1.name

'O1'

In [69]:
atom.element.symbol

'C'

In [39]:
data

,Name,Atom1,Atom2,Name1,Name2,Symbol1,Symbol2,r
0,HNI,3340,3341,NI,CHA,Ni,C,0.336
1,HNI,3340,3342,NI,CHB,Ni,C,0.336
2,HNI,3340,3343,NI,CHC,Ni,C,0.335
3,HNI,3340,3344,NI,CHD,Ni,C,0.337
4,HNI,3340,3345,NI,NA,Ni,N,0.193
...,...,...,...,...,...,...,...,...
3607,HNI,3508,3510,CBD,O1D,C,O,0.240
3608,HNI,3508,3511,CBD,O2D,C,O,0.250
3609,HNI,3509,3510,CGD,O1D,C,O,0.122
3610,HNI,3509,3511,CGD,O2D,C,O,0.135


In [40]:
data.to_csv(f"{pre}/het_frag.csv", index=False)

In [41]:
for i, line in data.iterrows():
    atom1 = line["Atom1"]
    atom2 = line["Atom2"]
    r = float(line["r"])
    print(atom1, atom2, r)
    

3340 3341 0.336
3340 3342 0.336
3340 3343 0.335
3340 3344 0.337
3340 3345 0.193
3340 3346 0.298
3340 3347 0.418
3340 3348 0.419
3340 3349 0.299
3340 3350 0.558
3340 3351 0.555
3340 3352 0.6
3340 3353 0.734
3340 3354 0.771
3340 3355 0.831
3340 3356 0.194
3340 3357 0.296
3340 3358 0.42
3340 3359 0.423
3340 3360 0.296
3340 3361 0.556
3340 3362 0.557
3340 3363 0.658
3340 3364 0.194
3340 3365 0.297
3340 3366 0.418
3340 3367 0.42
3340 3368 0.298
3340 3369 0.555
3340 3370 0.555
3340 3371 0.654
3340 3372 0.195
3340 3373 0.296
3340 3374 0.425
3340 3375 0.424
3340 3376 0.296
3340 3377 0.562
3340 3378 0.559
3340 3379 0.663
3340 3380 0.771
3340 3381 0.829
3340 3382 0.826
3341 3342 0.472
3341 3343 0.669
3341 3344 0.473
3341 3345 0.241
3341 3346 0.135
3341 3347 0.242
3341 3348 0.348
3341 3349 0.345
3341 3350 0.487
3341 3351 0.304
3341 3352 0.349
3341 3353 0.449
3341 3354 0.456
3341 3355 0.566
3341 3356 0.491
3341 3357 0.529
3341 3358 0.671
3341 3359 0.72
3341 3360 0.616
3341 3361 0.768
3341 3362 0.8

3355 3372 0.787
3355 3373 0.884
3355 3374 0.852
3355 3375 0.724
3355 3376 0.68
3355 3377 0.963
3355 3378 0.662
3355 3379 0.74
3355 3380 0.677
3355 3381 0.704
3355 3382 0.626
3356 3357 0.134
3356 3358 0.232
3356 3359 0.233
3356 3360 0.134
3356 3361 0.376
3356 3362 0.374
3356 3363 0.47
3356 3364 0.275
3356 3365 0.291
3356 3366 0.431
3356 3367 0.49
3356 3368 0.411
3356 3369 0.533
3356 3370 0.638
3356 3371 0.72
3356 3372 0.388
3356 3373 0.482
3356 3374 0.617
3356 3375 0.616
3356 3376 0.481
3356 3377 0.75
3356 3378 0.746
3356 3379 0.846
3356 3380 0.946
3356 3381 0.995
3356 3382 1.003
3357 3358 0.143
3357 3359 0.22
3357 3360 0.211
3357 3361 0.261
3357 3362 0.365
3357 3363 0.439
3357 3364 0.407
3357 3365 0.418
3357 3366 0.558
3357 3367 0.623
3357 3368 0.543
3357 3369 0.648
3357 3370 0.771
3357 3371 0.85
3357 3372 0.482
3357 3373 0.591
3357 3374 0.715
3357 3375 0.695
3357 3376 0.551
3357 3377 0.856
3357 3378 0.811
3357 3379 0.91
3357 3380 0.995
3357 3381 1.035
3357 3382 1.048
3358 3359 0.134
3

3390 3401 0.44
3390 3402 0.607
3390 3403 0.702
3390 3404 0.553
3390 3405 0.689
3390 3406 0.712
3390 3407 0.593
3390 3408 0.805
3390 3409 0.847
3390 3410 0.96
3390 3411 0.539
3390 3412 0.626
3390 3413 0.565
3390 3414 0.419
3390 3415 0.775
3390 3416 0.655
3390 3417 0.698
3390 3418 0.826
3390 3419 0.838
3390 3420 0.943
3390 3421 0.137
3390 3422 0.29
3390 3423 0.481
3390 3424 0.408
3390 3425 0.299
3391 3392 0.311
3391 3393 0.433
3391 3394 0.522
3391 3395 0.564
3391 3396 0.595
3391 3397 0.441
3391 3398 0.524
3391 3399 0.646
3391 3400 0.647
3391 3401 0.519
3391 3402 0.774
3391 3403 0.881
3391 3404 0.81
3391 3405 0.947
3391 3406 0.972
3391 3407 0.852
3391 3408 1.056
3391 3409 1.106
3391 3410 1.219
3391 3411 0.772
3391 3412 0.833
3391 3413 0.745
3391 3414 0.612
3391 3415 0.982
3391 3416 0.795
3391 3417 0.82
3391 3418 0.921
3391 3419 0.899
3391 3420 1.05
3391 3421 0.377
3391 3422 0.533
3391 3423 0.742
3391 3424 0.64
3391 3425 0.558
3392 3393 0.153
3392 3394 0.252
3392 3395 0.341
3392 3396 0.31


3427 3443 0.346
3427 3444 0.303
3427 3445 0.494
3427 3446 0.557
3427 3447 0.532
3427 3448 0.672
3427 3449 0.721
3427 3450 0.621
3427 3451 0.771
3427 3452 0.865
3427 3453 0.955
3427 3454 0.615
3427 3455 0.721
3427 3456 0.677
3427 3457 0.53
3427 3458 0.867
3427 3459 0.778
3427 3460 0.851
3427 3461 0.993
3427 3462 1.031
3427 3463 1.09
3427 3464 0.242
3427 3465 0.242
3427 3466 0.492
3427 3467 0.491
3427 3468 0.336
3428 3429 0.475
3428 3430 0.621
3428 3431 0.719
3428 3432 0.674
3428 3433 0.532
3428 3434 0.779
3428 3435 0.867
3428 3436 0.92
3428 3437 1.072
3428 3438 1.143
3428 3439 1.141
3428 3440 0.344
3428 3441 0.354
3428 3442 0.25
3428 3443 0.136
3428 3444 0.498
3428 3445 0.308
3428 3446 0.42
3428 3447 0.136
3428 3448 0.245
3428 3449 0.355
3428 3450 0.348
3428 3451 0.303
3428 3452 0.497
3428 3453 0.558
3428 3454 0.528
3428 3455 0.674
3428 3456 0.72
3428 3457 0.616
3428 3458 0.774
3428 3459 0.867
3428 3460 0.931
3428 3461 1.074
3428 3462 1.157
3428 3463 1.121
3428 3464 0.491
3428 3465 0.24

3448 3468 0.42
3449 3450 0.142
3449 3451 0.259
3449 3452 0.148
3449 3453 0.247
3449 3454 0.371
3449 3455 0.49
3449 3456 0.591
3449 3457 0.559
3449 3458 0.528
3449 3459 0.736
3449 3460 0.782
3449 3461 0.904
3449 3462 1.009
3449 3463 0.912
3449 3464 0.611
3449 3465 0.492
3449 3466 0.233
3449 3467 0.433
3449 3468 0.423
3450 3451 0.37
3450 3452 0.258
3450 3453 0.371
3450 3454 0.239
3450 3455 0.372
3450 3456 0.46
3450 3457 0.418
3450 3458 0.44
3450 3459 0.608
3450 3460 0.662
3450 3461 0.791
3450 3462 0.89
3450 3463 0.813
3450 3464 0.483
3450 3465 0.411
3450 3466 0.138
3450 3467 0.29
3450 3468 0.301
3451 3452 0.326
3451 3453 0.331
3451 3454 0.609
3451 3455 0.74
3451 3456 0.829
3451 3457 0.772
3451 3458 0.787
3451 3459 0.978
3451 3460 1.026
3451 3461 1.154
3451 3462 1.257
3451 3463 1.168
3451 3464 0.741
3451 3465 0.531
3451 3466 0.374
3451 3467 0.638
3451 3468 0.556
3452 3453 0.133
3452 3454 0.438
3452 3455 0.526
3452 3456 0.645
3452 3457 0.645
3452 3458 0.522
3452 3459 0.779
3452 3460 0.812


3482 3488 1.032
3482 3489 0.967
3482 3490 0.991
3482 3491 1.177
3482 3492 1.254
3482 3493 0.902
3482 3494 1.022
3482 3495 1.132
3482 3496 1.093
3482 3497 0.951
3482 3498 1.276
3482 3499 1.191
3482 3500 1.295
3482 3501 0.664
3482 3502 0.756
3482 3503 0.724
3482 3504 0.592
3482 3505 0.548
3482 3506 0.843
3482 3507 0.529
3482 3508 0.651
3482 3509 0.665
3482 3510 0.78
3482 3511 0.576
3483 3484 0.222
3483 3485 0.939
3483 3486 0.912
3483 3487 1.033
3483 3488 1.128
3483 3489 1.072
3483 3490 1.068
3483 3491 1.271
3483 3492 1.35
3483 3493 1.02
3483 3494 1.137
3483 3495 1.249
3483 3496 1.213
3483 3497 1.071
3483 3498 1.393
3483 3499 1.312
3483 3500 1.416
3483 3501 0.784
3483 3502 0.877
3483 3503 0.841
3483 3504 0.708
3483 3505 0.669
3483 3506 0.956
3483 3507 0.636
3483 3508 0.748
3483 3509 0.745
3483 3510 0.856
3483 3511 0.642
3484 3485 0.847
3484 3486 0.849
3484 3487 0.984
3484 3488 1.06
3484 3489 0.979
3484 3490 1.049
3484 3491 1.207
3484 3492 1.281
3484 3493 0.875
3484 3494 1.003
3484 3495 1.

In [16]:
len(positions)

3760

In [52]:
pos1 = positions[3548]

In [53]:
pos2 = positions[3551]

In [54]:
(pos1[0]**2 + pos1[1]**2 + pos1[2]**2)**0.5

Quantity(value=1.7304134910477322, unit=nanometer)

In [55]:
dis = pos1 - pos2
dis = dis.value_in_unit(nanometer)
r = (dis[0]**2 + dis[1]**2 + dis[2]**2)**0.5

In [56]:
r

0.15375965010366024

In [17]:
res = ligand_res_list[0]

In [18]:
b = list(res.bonds())[0]

In [19]:
b.atom1

<Atom 3548 (C18) of chain 5 residue 604 (OLC)>

In [20]:
list(res.bonds())

[Bond(<Atom 3548 (C18) of chain 5 residue 604 (OLC)>, <Atom 3551 (C17) of chain 5 residue 604 (OLC)>),
 Bond(<Atom 3549 (C10) of chain 5 residue 604 (OLC)>, <Atom 3550 (C9) of chain 5 residue 604 (OLC)>),
 Bond(<Atom 3549 (C10) of chain 5 residue 604 (OLC)>, <Atom 3552 (C11) of chain 5 residue 604 (OLC)>),
 Bond(<Atom 3550 (C9) of chain 5 residue 604 (OLC)>, <Atom 3553 (C8) of chain 5 residue 604 (OLC)>),
 Bond(<Atom 3551 (C17) of chain 5 residue 604 (OLC)>, <Atom 3555 (C16) of chain 5 residue 604 (OLC)>),
 Bond(<Atom 3552 (C11) of chain 5 residue 604 (OLC)>, <Atom 3556 (C12) of chain 5 residue 604 (OLC)>),
 Bond(<Atom 3553 (C8) of chain 5 residue 604 (OLC)>, <Atom 3557 (C7) of chain 5 residue 604 (OLC)>),
 Bond(<Atom 3554 (C24) of chain 5 residue 604 (OLC)>, <Atom 3568 (C22) of chain 5 residue 604 (OLC)>),
 Bond(<Atom 3554 (C24) of chain 5 residue 604 (OLC)>, <Atom 3570 (O25) of chain 5 residue 604 (OLC)>),
 Bond(<Atom 3555 (C16) of chain 5 residue 604 (OLC)>, <Atom 3558 (C15) of chai

In [160]:
b.type

In [161]:
atom1 = b.atom1

In [162]:
atom1.id

'31351'

In [163]:
atom1.index

3548

In [168]:
atom1.element.symbol

'C'

In [ ]:
# initial testing.

In [5]:
origin_pdb = PDBFile("/Users/weilu/Research/server/jan_2020/include_small_molecular/6ud8_F_complete/6ud8_F_complete-openmmawsem.pdb")

In [6]:
origin_pdb.getTopology()

<Topology; 1 chains, 174 residues, 1028 atoms, 0 bonds>

In [64]:
pdb = PDBFile("/Users/weilu/Research/server/jan_2020/include_small_molecular/6ud8_F_complete-openmmawsem.pdb")

In [65]:
pdb.getTopology()

<Topology; 2 chains, 175 residues, 1093 atoms, 64 bonds>

In [66]:
forcefield = ForceField("/Users/weilu/openmmawsem/awsem.xml")

In [67]:
forcefield.generateTemplatesForUnmatchedResidues(pdb.topology)

[[<simtk.openmm.app.forcefield.ForceField._TemplateData at 0x1a20bc71d0>],
 [<Residue 174 (OLC) of chain 1>]]

In [68]:
[template, ffxml] = forcefield.generateTemplatesForUnmatchedResidues(pdb.topology)

In [69]:
a = template[0]

In [70]:
a.name

'OLC'

In [71]:
a.bonds

[(0, 3),
 (0, 25),
 (0, 26),
 (0, 27),
 (1, 2),
 (1, 4),
 (1, 28),
 (2, 5),
 (2, 29),
 (3, 7),
 (3, 30),
 (3, 31),
 (4, 8),
 (4, 32),
 (4, 33),
 (5, 9),
 (5, 34),
 (5, 35),
 (6, 20),
 (6, 22),
 (6, 36),
 (6, 37),
 (7, 10),
 (7, 38),
 (7, 39),
 (8, 11),
 (8, 40),
 (8, 41),
 (9, 12),
 (9, 42),
 (9, 43),
 (10, 13),
 (10, 44),
 (10, 45),
 (11, 13),
 (11, 46),
 (11, 47),
 (12, 14),
 (12, 48),
 (12, 49),
 (13, 50),
 (13, 51),
 (14, 15),
 (14, 52),
 (14, 53),
 (15, 16),
 (15, 54),
 (15, 55),
 (16, 17),
 (16, 56),
 (16, 57),
 (17, 19),
 (17, 58),
 (17, 59),
 (18, 20),
 (18, 24),
 (18, 60),
 (18, 61),
 (19, 21),
 (19, 24),
 (20, 23),
 (20, 62),
 (22, 63),
 (23, 64)]

In [56]:
a

In [86]:
for a1 in a.atoms:
    a1.type = a1.element.symbol

In [72]:
a1 = a.atoms[0]

In [73]:
a1.element.symbol

'C'

In [74]:
a1.name

'C18'

In [75]:
a1.parameters

{}

In [76]:
a1.type

In [77]:
a1.externalBonds

0

In [78]:
a.externalBonds

[]

In [80]:
a1.bondedTo

[3, 25, 26, 27]

In [82]:
a.atoms[3].name

'C17'

In [21]:
olc = a[1]

In [40]:
forcefield.getGenerators()

[]

In [84]:
forcefield.registerResidueTemplate(a)

In [89]:
system = forcefield.createSystem(pdb.topology)

In [90]:
system.getNumParticles()

1093

In [126]:
system.

<Swig Object of type 'OpenMM::System *' at 0x1a21f8a600>

In [95]:
b = list(pdb.topology.atoms())

In [96]:
b[-1]

<Atom 1092 (HO23) of chain 1 residue 174 (OLC)>

In [97]:
b[0]

<Atom 0 (CA) of chain 0 residue 0 (NGP)>

In [103]:
b0 = b[-1]

In [104]:
b0.id

'31415'

In [105]:
b0.index

1092

In [106]:
b0.element

<Element hydrogen>

In [107]:
pdb.topology.getNumResidues()

175

In [111]:
res_list = list(pdb.topology.residues())
protein_resNames = ["NGP", "IGL", "IPR", "NTER", "CTER"]
DNA_resNames = ["DA", "DC", "DT", "DG"]
protein_res_list = []
DNA_res_list = []
ligand_res_list = []
for res in res_list:
    if res.name in protein_resNames:
        protein_res_list.append(res)
    elif res.name in DNA_resNames:
        DNA_res_list.append(res)
    else:
        ligand_res_list.append(res)

In [112]:
res = res_list[0]

In [ ]:
nres = len(protein_res_list)
residues = protein_res_list
natoms 

In [117]:
atom_list = list(pdb.topology.atoms())

In [124]:
protein_atom_list = []
DNA_atom_list = []
ligand_atom_list = []
for atom in atom_list:
    if atom.residue.name in protein_resNames:
        protein_atom_list.append(atom)
    elif atom.residue.name in DNA_resNames:
        DNA_atom_list.append(atom)
    else:
        ligand_atom_list.append(atom)

In [119]:
atom = atom_list[0]

In [125]:
ligand_atom_list

[<Atom 1028 (C18) of chain 1 residue 174 (OLC)>,
 <Atom 1029 (C10) of chain 1 residue 174 (OLC)>,
 <Atom 1030 (C9) of chain 1 residue 174 (OLC)>,
 <Atom 1031 (C17) of chain 1 residue 174 (OLC)>,
 <Atom 1032 (C11) of chain 1 residue 174 (OLC)>,
 <Atom 1033 (C8) of chain 1 residue 174 (OLC)>,
 <Atom 1034 (C24) of chain 1 residue 174 (OLC)>,
 <Atom 1035 (C16) of chain 1 residue 174 (OLC)>,
 <Atom 1036 (C12) of chain 1 residue 174 (OLC)>,
 <Atom 1037 (C7) of chain 1 residue 174 (OLC)>,
 <Atom 1038 (C15) of chain 1 residue 174 (OLC)>,
 <Atom 1039 (C13) of chain 1 residue 174 (OLC)>,
 <Atom 1040 (C6) of chain 1 residue 174 (OLC)>,
 <Atom 1041 (C14) of chain 1 residue 174 (OLC)>,
 <Atom 1042 (C5) of chain 1 residue 174 (OLC)>,
 <Atom 1043 (C4) of chain 1 residue 174 (OLC)>,
 <Atom 1044 (C3) of chain 1 residue 174 (OLC)>,
 <Atom 1045 (C2) of chain 1 residue 174 (OLC)>,
 <Atom 1046 (C21) of chain 1 residue 174 (OLC)>,
 <Atom 1047 (C1) of chain 1 residue 174 (OLC)>,
 <Atom 1048 (C22) of chain 1 

In [120]:
atom.index

0

In [121]:
atom.name

'CA'

In [123]:
atom.residue.name

'NGP'

In [116]:
DNA_res_list

[]

In [115]:
ligand_res_list

[<Residue 174 (OLC) of chain 1>]

In [ ]:
for res in res_list:
    if 